In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, classification_report

#定义数据路径和输出文件路径
dpath = 'D:\\Python_study\\5组代码\\卒中发病预测\\'
outfile = dpath + 'output__new/Stroke___panel/Stroke_hierarchy.csv'
df = pd.read_csv("D:\\课题\\peo_people448749.csv")
# 确保 data_imputed 是 DataFrame
data_imputed_df = pd.read_csv("D:\\Python_study\\5组代码\\卒中发病预测\\peo_imputed_data448749(mean).csv")

In [3]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
import time
import joblib
# 筛选来自英格兰的参与者
england_indices = df[df['Region'] == 'England'].index
X_england = data_imputed_df.loc[england_indices]
y_england = df.loc[england_indices, 'status']

# 定义5折交叉验证
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# 存储每折的评估结果
accuracy_scores = []
auc_scores = []
training_times = []
models = []
# 计算类别权重（需要在循环内计算，因为每折的分布可能不同）
# 注意：这里我们会在循环内计算，因为y_train在每折不同

# 固定参数（从你的网格中取第一个值）
params = {
    'alpha': 1.5,          # L1 正则化
    'lambda': 1.5,         # L2 正则化
    'gamma': 0.2,          # 分裂最小损失下降
    'colsample_bytree': 0.75,  # 特征采样比例
    'subsample': 0.85,     # 数据采样比例
    'learning_rate': 0.007, # 学习率
    'max_depth': 9,        # 树的最大深度
    'n_estimators': 5200,  # 树的数量
    'min_child_weight': 3, # 叶子节点最小样本权重
    'eval_metric':'auc'
}
# 开始交叉验证
for fold, (train_idx, val_idx) in enumerate(kf.split(X_england), 1):
    try:
        print(f"\n===== Fold {fold} =====\n")
        X_train, X_val = X_england.iloc[train_idx], X_england.iloc[val_idx]
        y_train, y_val = y_england.iloc[train_idx], y_england.iloc[val_idx]

        # 计算当前fold的类别权重
        scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)
        current_params = params.copy()
        current_params['scale_pos_weight'] = scale_pos_weight

        # 初始化模型
        model = XGBClassifier(**current_params)

        # 训练模型并计时
        start_time = time.time()
        print("Training started...")
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=10  # 每10轮显示一次进度
        )
        training_time = time.time() - start_time
        training_times.append(training_time)
        print(f"Training completed in {training_time:.2f} seconds")

        # 预测
        y_pred = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_val)[:, 1]  # 获取正类的概率

        # 评估
        accuracy = accuracy_score(y_val, y_pred)
        auc = roc_auc_score(y_val, y_pred_proba)

        accuracy_scores.append(accuracy)
        auc_scores.append(auc)
        models.append(model)  # 保存当前fold的模型
        print(f"Fold {fold} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold} AUC: {auc:.4f}")

    except KeyboardInterrupt:
        print("\nTraining interrupted by user")
        break
    except Exception as e:
        print(f"\nError occurred during fold {fold}: {str(e)}")
        continue

# 输出结果
if accuracy_scores:
    print("\n=== Final Results ===")
    print(f"Average Accuracy: {np.mean(accuracy_scores):.4f} ± {np.std(accuracy_scores):.4f}")
    print(f"Average AUC: {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")
    print(f"Average Training Time: {np.mean(training_times):.2f} seconds")

    # 找到AUC最高的fold
    best_fold = np.argmax(auc_scores)
    best_model = models[best_fold]
    best_auc = auc_scores[best_fold]

    print(f"\nBest model from Fold {best_fold + 1} with AUC: {best_auc:.4f}")

    # 保存最佳模型
    model_filename = f"best_xgboost_model_fold{best_fold + 1}_auc{best_auc:.4f}.joblib"
    # joblib.dump(best_model, model_filename)
    print(f"Best model saved as {model_filename}")
else:
    print("\nNo valid results were obtained")


===== Fold 1 =====

Training started...
[0]	validation_0-auc:0.65544
[10]	validation_0-auc:0.70464
[20]	validation_0-auc:0.70877
[30]	validation_0-auc:0.71047
[40]	validation_0-auc:0.71158
[50]	validation_0-auc:0.71282
[60]	validation_0-auc:0.71355
[70]	validation_0-auc:0.71420
[80]	validation_0-auc:0.71483
[90]	validation_0-auc:0.71448
[100]	validation_0-auc:0.71464
[110]	validation_0-auc:0.71465
[120]	validation_0-auc:0.71493
[130]	validation_0-auc:0.71525
[140]	validation_0-auc:0.71547
[150]	validation_0-auc:0.71545
[160]	validation_0-auc:0.71549
[170]	validation_0-auc:0.71578
[180]	validation_0-auc:0.71612
[190]	validation_0-auc:0.71645
[200]	validation_0-auc:0.71648
[210]	validation_0-auc:0.71622
[220]	validation_0-auc:0.71628
[230]	validation_0-auc:0.71634
[240]	validation_0-auc:0.71616
[250]	validation_0-auc:0.71616
[260]	validation_0-auc:0.71616
[270]	validation_0-auc:0.71604
[280]	validation_0-auc:0.71597
[290]	validation_0-auc:0.71598
[300]	validation_0-auc:0.71604
[310]	val

In [4]:
external_indices = df[df['Region'].isin(['Scotland', 'Wales'])].index
# 获取苏格兰和威尔士参与者的数据
X_external = data_imputed_df.loc[external_indices]  # 特征
y_external = df.loc[external_indices, 'status']  # 标签
print("外部验证集形状:", X_external.shape)
X_external
# 预测类别（0/1）
y_pred = best_model.predict(X_external)
# 预测概率（用于 AUC 等指标）
y_pred_proba = best_model.predict_proba(X_external)[:, 1]  # 正类的概率
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

# 计算准确率和 AUC
accuracy = accuracy_score(y_external, y_pred)
auc = roc_auc_score(y_external, y_pred_proba)

print(f"测试集准确率: {accuracy:.4f}")
print(f"测试集 AUC: {auc:.4f}")

外部验证集形状: (50653, 194)
测试集准确率: 0.9567
测试集 AUC: 0.6826


In [6]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
import time
import joblib
# 筛选来自英格兰的参与者
england_indices = df[df['Region'] == 'England'].index
X_england = data_imputed_df.loc[england_indices]
y_england = df.loc[england_indices, 'status']

# 定义5折交叉验证
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# 存储每折的评估结果
accuracy_scores = []
auc_scores = []
training_times = []
models = []
# 计算类别权重（需要在循环内计算，因为每折的分布可能不同）
# 注意：这里我们会在循环内计算，因为y_train在每折不同

# 固定参数（从你的网格中取第一个值）
params = {
    'alpha': 1.5,          # L1 正则化
    'lambda': 1.5,         # L2 正则化
    'colsample_bytree': 0.75,  # 特征采样比例
    'n_estimators': 400,
    'learning_rate': 0.01,
    'max_depth': 3,
    'subsample': 0.7,
    'min_child_weight': 3,
    'gamma': 0.9,
    'eval_metric':'auc'
}
# 开始交叉验证
for fold, (train_idx, val_idx) in enumerate(kf.split(X_england), 1):
    try:
        print(f"\n===== Fold {fold} =====\n")
        X_train, X_val = X_england.iloc[train_idx], X_england.iloc[val_idx]
        y_train, y_val = y_england.iloc[train_idx], y_england.iloc[val_idx]

        # 计算当前fold的类别权重
        scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)
        current_params = params.copy()
        current_params['scale_pos_weight'] = scale_pos_weight

        # 初始化模型
        model = XGBClassifier(**current_params)

        # 训练模型并计时
        start_time = time.time()
        print("Training started...")
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=10  # 每10轮显示一次进度
        )
        training_time = time.time() - start_time
        training_times.append(training_time)
        print(f"Training completed in {training_time:.2f} seconds")

        # 预测
        y_pred = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_val)[:, 1]  # 获取正类的概率

        # 评估
        accuracy = accuracy_score(y_val, y_pred)
        auc = roc_auc_score(y_val, y_pred_proba)

        accuracy_scores.append(accuracy)
        auc_scores.append(auc)
        models.append(model)  # 保存当前fold的模型
        print(f"Fold {fold} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold} AUC: {auc:.4f}")

    except KeyboardInterrupt:
        print("\nTraining interrupted by user")
        break
    except Exception as e:
        print(f"\nError occurred during fold {fold}: {str(e)}")
        continue

# 输出结果
if accuracy_scores:
    print("\n=== Final Results ===")
    print(f"Average Accuracy: {np.mean(accuracy_scores):.4f} ± {np.std(accuracy_scores):.4f}")
    print(f"Average AUC: {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")
    print(f"Average Training Time: {np.mean(training_times):.2f} seconds")

    # 找到AUC最高的fold
    best_fold = np.argmax(auc_scores)
    best_model = models[best_fold]
    best_auc = auc_scores[best_fold]

    print(f"\nBest model from Fold {best_fold + 1} with AUC: {best_auc:.4f}")

    # 保存最佳模型
    model_filename = f"best_xgboost_model_fold{best_fold + 1}_auc{best_auc:.4f}.joblib"
    # joblib.dump(best_model, model_filename)
    print(f"Best model saved as {model_filename}")
else:
    print("\nNo valid results were obtained")


===== Fold 1 =====

Training started...
[0]	validation_0-auc:0.68671
[10]	validation_0-auc:0.70096
[20]	validation_0-auc:0.70281
[30]	validation_0-auc:0.70396
[40]	validation_0-auc:0.70550
[50]	validation_0-auc:0.70669
[60]	validation_0-auc:0.70728
[70]	validation_0-auc:0.70811
[80]	validation_0-auc:0.70883
[90]	validation_0-auc:0.70975
[100]	validation_0-auc:0.71047
[110]	validation_0-auc:0.71139
[120]	validation_0-auc:0.71206
[130]	validation_0-auc:0.71279
[140]	validation_0-auc:0.71315
[150]	validation_0-auc:0.71390
[160]	validation_0-auc:0.71432
[170]	validation_0-auc:0.71493
[180]	validation_0-auc:0.71532
[190]	validation_0-auc:0.71584
[200]	validation_0-auc:0.71619
[210]	validation_0-auc:0.71676
[220]	validation_0-auc:0.71715
[230]	validation_0-auc:0.71746
[240]	validation_0-auc:0.71773
[250]	validation_0-auc:0.71792
[260]	validation_0-auc:0.71832
[270]	validation_0-auc:0.71859
[280]	validation_0-auc:0.71880
[290]	validation_0-auc:0.71907
[300]	validation_0-auc:0.71926
[310]	val

In [7]:
external_indices = df[df['Region'].isin(['Scotland', 'Wales'])].index
# 获取苏格兰和威尔士参与者的数据
X_external = data_imputed_df.loc[external_indices]  # 特征
y_external = df.loc[external_indices, 'status']  # 标签
print("外部验证集形状:", X_external.shape)
X_external
# 预测类别（0/1）
y_pred = best_model.predict(X_external)
# 预测概率（用于 AUC 等指标）
y_pred_proba = best_model.predict_proba(X_external)[:, 1]  # 正类的概率
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

# 计算准确率和 AUC
accuracy = accuracy_score(y_external, y_pred)
auc = roc_auc_score(y_external, y_pred_proba)

print(f"测试集准确率: {accuracy:.4f}")
print(f"测试集 AUC: {auc:.4f}")

外部验证集形状: (50653, 194)
测试集准确率: 0.6486
测试集 AUC: 0.7344


In [5]:
# 4. 用最佳模型预测
y_proba = best_model.predict_proba(X_val)[:, 1]
# 5. 计算并绘制 ROC 曲线
fpr, tpr, _ = roc_curve(y_val, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Best XGBoost ROC Curve(internal)')
plt.legend()
plt.show()
#绘制混淆矩阵
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 使用最佳模型进行预测
y_pred = best_model.predict(X_val)

# 计算混淆矩阵
cm = confusion_matrix(y_val, y_pred)

# 可视化混淆矩阵
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['No Stroke', 'Stroke'])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix for Stroke Prediction(internal)')
plt.show()
#外部 4. 用最佳模型预测
y_proba = best_model.predict_proba(X_external)[:, 1]
# 5. 计算并绘制 ROC 曲线
fpr, tpr, _ = roc_curve(y_external, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Best XGBoost ROC Curve(external)')
plt.legend()
plt.show()
#绘制混淆矩阵
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 使用最佳模型进行预测
y_pred = best_model.predict(X_external)

# 计算混淆矩阵
cm = confusion_matrix(y_external, y_pred)

# 可视化混淆矩阵
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['No Stroke', 'Stroke'])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix for Stroke Prediction(external)')
plt.show()

TypeError: 'numpy.float64' object is not callable